<a href="https://colab.research.google.com/github/AphiphopSrisueb/ET_Porshe/blob/main/ET_Porsche.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import hashlib

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
%matplotlib inline
import time
import numpy as np

In [ ]:
from PIL import Image

In [ ]:
import os

In [ ]:
def file_hash(filepath): # create hash value of each images
    with open(filepath, 'rb') as f:
        return md5(f.read()).hexdigest()

In [ ]:
os.chdir(r'/content/drive/MyDrive/ET_Porshe/train') # path you want to remove duplicates
os.getcwd()

In [ ]:
os.getcwd() # check path

In [ ]:
file_list = os.listdir() # check amount of files in folder
print(len(file_list))

22


In [ ]:
import hashlib, os # find duplicates
duplicates = []
hash_keys = dict()
for index, filename in  enumerate(os.listdir('.')):  #listdir('.') = current directory
    if os.path.isfile(filename):
        with open(filename, 'rb') as f:
            filehash = hashlib.md5(f.read()).hexdigest()
        if filehash not in hash_keys:
            hash_keys[filehash] = index
        else:
            duplicates.append((index,hash_keys[filehash]))

In [ ]:
duplicates # check if there is duplicates or not

In [ ]:
for file_indexes in duplicates[:30]: # if there is duplicates you can check the image that is duplicates
    try:

        plt.subplot(121),plt.imshow(Image.open(file_list[file_indexes[1]]))
        plt.title(file_indexes[1]), plt.xticks([]), plt.yticks([])

        plt.subplot(122),plt.imshow(Image.open(file_list[file_indexes[0]]))
        plt.title(str(file_indexes[0]) + ' duplicate'), plt.xticks([]), plt.yticks([])
        plt.show()

    except OSError as e:
        continue

TRAINING TIME

In [ ]:
!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

In [ ]:
from fastbook import *

In [ ]:
# Define your data block
fields = DataBlock(
    blocks=(ImageBlock, CategoryBlock),
    get_items=get_image_files,
    get_y=parent_label,
    splitter=GrandparentSplitter(train_name='train', valid_name='vali'),  # Split into train and validation
    item_tfms=RandomResizedCrop(224, min_scale=0.5),  # Resize images
    batch_tfms=aug_transforms()
)

In [ ]:
paths = get_image_files('/content/gdrive/MyDrive/ET_Porshe/') # work folder that contain data

In [ ]:
dls = fields.dataloaders("/content/gdrive/MyDrive/ET_Porshe/") # dataloader

In [ ]:
dls.train.show_batch(max_n=8, nrows=2) # check that image and data correspond or not

In [ ]:
dls.vocab # check all classes

In [ ]:
learner = cnn_learner(dls, resnet34, metrics=[error_rate, accuracy]) # setup a learner

In [ ]:
learner.lr_find() # find the learning rate

In [ ]:
learner.fine_tune(epochs=6, freeze_epochs=1, base_lr=0.0012) # fine tune the model for 6 epochs by freezing the first epoch

In [ ]:
interp = ClassificationInterpretation.from_learner(learner)
interp.plot_confusion_matrix(normalize=True, norm_dec=2, figsize=(22,22), dpi=60) # plot a confusion matrix to look at the accuracy of each classes

In [ ]:
learner.show_results(shuffle=True) # see the result of the model

In [ ]:
learner.export("PORSCHE MD") # save your model